In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
from __future__ import print_function

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [3]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [5]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [6]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

model.load('./model.tflearn')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'healthproblems_me', u'healthproblems_others', u'jobloss_me', u'jobloss_others', u'naturaldisasters_me', u'naturaldisasters_others']
INFO:tensorflow:Restoring parameters from /home/ignitelab/Desktop/app/model.tflearn


In [7]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                   if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                   if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return print(random.choice(i['responses']))
                        #return random.choice(i['responses'])

            results.pop(0)

In [8]:
classify('i lost job')


[(u'jobloss_me', 0.75159341)]

In [9]:
response('my friend quit job')

jobloss_others_NONE


In [10]:
response('i am suffering from lungs disesase')

healthproblems_me_HIGH


In [11]:
response('suffering from lungs disesase')

healthproblems_me_HIGH


In [12]:
response('i lost my job..feeling bad')

jobloss_others_NONE


In [13]:
response('i had a heart attack yesterday. stable now')

healthproblems_me_HIGH
